In [2]:
import pandas as pd 

In [10]:
posts_df=pd.read_csv('combined_no_duplicates_post.csv')
print(posts_df)

                                              Nom_poste  \
0                                DBA SQL/Big Data (H/F)   
1                                    Data analyst (H/F)   
2     Contrôle de gestion données sociales RH/Data A...   
3                      Comptable /Data contrôleur (h/f)   
4                     Ingénieur Data GCP Confirmé (H/F)   
...                                                 ...   
1530  Alternance Ingénieur Data analyst - Munster (H/F)   
1531  Alternance Data analyst/testeur - Paris La Déf...   
1532   Alternance Data analyst - Toussus-le-Noble (H/F)   
1533                  Data Analyst - Biomasse - CDD H/F   
1534                   Stream leader data ibp f/h (H/F)   

                                                Contrat  \
0                            CDI\r\n\r\nContrat travail   
1              Intérim - 18 Mois\r\n\r\nContrat travail   
2                            CDI\r\n\r\nContrat travail   
3                            CDI\r\n\r\nContrat travail

In [11]:
posts_df['full_text_job'] = (
    "[TITRE] " + posts_df['Nom_poste'].fillna('') + '\n' +
    "[CONTRAT] " + posts_df['Contrat'].fillna('') + '\n' +
    "[DESCRIPTION] " + posts_df['Description'].fillna('') + '\n' +
    "[SALAIRE] " + posts_df['Salaire'].fillna('') + '\n' +
    "[EXPERIENCE] " + posts_df['Experience'].fillna('') + '\n' +
    "[ENTREPRISE] " + posts_df['Entreprise'].fillna('')
)

posts_df[['Nom_poste', 'full_text_job']].head(3)



,Nom_poste,full_text_job
0,DBA SQL/Big Data (H/F),[TITRE] DBA SQL/Big Data (H/F)\n[CONTRAT] CDI\...
1,Data analyst (H/F),[TITRE] Data analyst (H/F)\n[CONTRAT] Intérim ...
2,Contrôle de gestion données sociales RH/Data A...,[TITRE] Contrôle de gestion données sociales R...


In [12]:
candidates_df = pd.read_csv("combined_no_duplicates.csv")

candidates_df['full_text_profile'] = (
    "[PROFIL] " + candidates_df['Profil'].fillna('') + '\n' +
    "[POINTS_FORTS] " + candidates_df['Points_forts'].fillna('') + '\n' +
    "[EXPERIENCE] " + candidates_df['Expérience'].fillna('') + '\n' +
    "[COMPETENCES] " + candidates_df['Compétence'].fillna('')
)

candidates_df[['Profil', 'full_text_profile']].head(3)

,Profil,full_text_profile
0,Ingénieur en sécurité et administration systè...,[PROFIL] Ingénieur en sécurité et administrat...
1,Ingénieur IT Junior – Development | Data | BI ...,[PROFIL] Ingénieur IT Junior – Development | D...
2,Développeur Python spécialisé en Data Engineering,[PROFIL] Développeur Python spécialisé en Data...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

job_texts = posts_df['full_text_job'].fillna('').tolist()
profile_texts = candidates_df['full_text_profile'].fillna('').tolist()

vectorizer = TfidfVectorizer()
all_texts = job_texts + profile_texts
tfidf_matrix = vectorizer.fit_transform(all_texts)

tfidf_jobs = tfidf_matrix[:len(job_texts)]
tfidf_profiles = tfidf_matrix[len(job_texts):]

similarity_matrix = cosine_similarity(tfidf_jobs, tfidf_profiles)

top_matches = []
for job_idx, similarities in enumerate(similarity_matrix):
    top_indices = similarities.argsort()[-3:][::-1] 
    for rank, profile_idx in enumerate(top_indices, start=1):
        top_matches.append({
            'Nom_poste': posts_df.iloc[job_idx]['Nom_poste'],
            'Profil_candidat': candidates_df.iloc[profile_idx]['Profil'],
            'Score_similarité': round(similarities[profile_idx], 3),
            'Rang': rank
        })

top_matches_df = pd.DataFrame(top_matches)
print(top_matches_df)


                              Nom_poste  \
0                DBA SQL/Big Data (H/F)   
1                DBA SQL/Big Data (H/F)   
2                DBA SQL/Big Data (H/F)   
3                    Data analyst (H/F)   
4                    Data analyst (H/F)   
...                                 ...   
4600  Data Analyst - Biomasse - CDD H/F   
4601  Data Analyst - Biomasse - CDD H/F   
4602   Stream leader data ibp f/h (H/F)   
4603   Stream leader data ibp f/h (H/F)   
4604   Stream leader data ibp f/h (H/F)   

                                        Profil_candidat  Score_similarité  \
0     A la recherche d'un emploi dans le Business In...             0.251   
1            Directeur des Systèmes d'information (DSI)             0.228   
2                                        Data Scientist             0.216   
3             chargé d'étude statistique/Data scientist             0.363   
4     Développeur Python, Expert en Science des Donn...             0.318   
...                  

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Chargement des fichiers
posts_df = pd.read_csv('combined_no_duplicates_post.csv')
candidates_df = pd.read_csv('combined_no_duplicates.csv')

# Création du texte structuré pour chaque offre
posts_df['full_text_job'] = (
    "[TITRE] " + posts_df['Nom_poste'].fillna('') + '\n' +
    "[CONTRAT] " + posts_df['Contrat'].fillna('') + '\n' +
    "[DESCRIPTION] " + posts_df['Description'].fillna('') + '\n' +
    "[SALAIRE] " + posts_df['Salaire'].fillna('') + '\n' +
    "[EXPERIENCE] " + posts_df['Experience'].fillna('') + '\n' +
    "[ENTREPRISE] " + posts_df['Entreprise'].fillna('')
)

# Création du texte structuré pour chaque candidat
candidates_df['full_text_profile'] = (
    "[PROFIL] " + candidates_df['Profil'].fillna('') + '\n' +
    "[POINTS_FORTS] " + candidates_df['Points_forts'].fillna('') + '\n' +
    "[EXPERIENCE] " + candidates_df['Expérience'].fillna('') + '\n' +
    "[COMPETENCES] " + candidates_df['Compétence'].fillna('')
)

# Chargement du modèle multilingue
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Encodage des textes
job_texts = posts_df['full_text_job'].tolist()
profile_texts = candidates_df['full_text_profile'].tolist()

job_embeddings = model.encode(job_texts, show_progress_bar=True)
profile_embeddings = model.encode(profile_texts, show_progress_bar=True)

# Similarité cosinus
similarity_matrix = cosine_similarity(job_embeddings, profile_embeddings)

# Top 3 matching
top_matches = []
for job_idx, similarities in enumerate(similarity_matrix):
    top_indices = similarities.argsort()[-3:][::-1]
    for rank, profile_idx in enumerate(top_indices, start=1):
        top_matches.append({
            'Nom_poste': posts_df.iloc[job_idx]['Nom_poste'],
            'Profil_candidat': candidates_df.iloc[profile_idx]['Profil'],
            'Score_similarité': round(similarities[profile_idx], 3),
            'Rang': rank
        })

top_matches_df = pd.DataFrame(top_matches)
print(top_matches_df.head(10))


ModuleNotFoundError: No module named 'pandas'